# Python Task 2
Question 1: Distance Matrix Calculation
Create a function named calculate_distance_matrix that takes the dataset-3.csv as input and generates a DataFrame representing distances between IDs.

The resulting DataFrame should have cumulative distances along known routes, with diagonal values set to 0. If distances between toll locations A to B and B to C are known, then the distance from A to C should be the sum of these distances. Ensure the matrix is symmetric, accounting for bidirectional distances between toll locations (i.e. A to B is equal to B to A).

In [43]:
def calculate_distance_matrix(dataset_path):
    
    dataset = pd.read_csv(dataset_path)

    unique_ids = sorted(set(dataset['id_start'].unique()) | set(dataset['id_end'].unique()))

    distance_matrix = pd.DataFrame(index=unique_ids, columns=unique_ids).fillna(0)

    for _, row in dataset.iterrows():
        distance_matrix.at[row['id_start'], row['id_end']] += row['distance']

    diagonal_values = distance_matrix.values.diagonal()
    distance_matrix = distance_matrix + distance_matrix.T - np.diag(diagonal_values)

    return distance_matrix

dataset_path = "D:\dataset-3.csv"
result_distance_matrix = calculate_distance_matrix(dataset_path)
print(result_distance_matrix)


         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7      0.0      0.0      0.0      0.0      0.0   
1001402      9.7      0.0     20.2      0.0      0.0      0.0      0.0   
1001404      0.0     20.2      0.0     16.0      0.0      0.0      0.0   
1001406      0.0      0.0     16.0      0.0     21.7      0.0      0.0   
1001408      0.0      0.0      0.0     21.7      0.0     11.1      0.0   
1001410      0.0      0.0      0.0      0.0     11.1      0.0     15.6   
1001412      0.0      0.0      0.0      0.0      0.0     15.6      0.0   
1001414      0.0      0.0      0.0      0.0      0.0      0.0     18.2   
1001416      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001418      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001420      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001422      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001424      0.0      0.0      0.0    

# Question 2: Unroll Distance Matrix
Create a function unroll_distance_matrix that takes the DataFrame created in Question 1. The resulting DataFrame should have three columns: columns id_start, id_end, and distance.

All the combinations except for same id_start to id_end must be present in the rows with their distance values from the input DataFrame.

In [44]:
import pandas as pd
import numpy as np

def unroll_distance_matrix(distance_matrix):
    distance_matrix = distance_matrix + distance_matrix.T

    mask = ~pd.DataFrame(np.eye(len(distance_matrix), dtype=bool), index=distance_matrix.index, columns=distance_matrix.columns)

    unrolled_distances = distance_matrix.stack().reset_index()

    unrolled_distances.columns = ['id_start', 'id_end', 'distance']

    unrolled_distances = unrolled_distances[mask.stack().values].reset_index(drop=True)

    return unrolled_distances

unrolled_distances = unroll_distance_matrix(result_distance_matrix)
print(unrolled_distances)


      id_start   id_end  distance
0      1001400  1001402      19.4
1      1001400  1001404       0.0
2      1001400  1001406       0.0
3      1001400  1001408       0.0
4      1001400  1001410       0.0
...        ...      ...       ...
1801   1004356  1001470       0.0
1802   1004356  1001472       0.0
1803   1004356  1001488       8.0
1804   1004356  1004354       4.0
1805   1004356  1004355       0.0

[1806 rows x 3 columns]


# Question 3: Finding IDs within Percentage Threshold¶
Create a function find_ids_within_ten_percentage_threshold that takes the DataFrame created in Question 2 and a reference value from the id_start column as an integer.

Calculate average distance for the reference value given as an input and return a sorted list of values from id_start column which lie within 10% (including ceiling and floor) of the reference value's average.

In [45]:
import pandas as pd
import numpy as np

def find_ids_within_ten_percentage_threshold(unrolled_distances, reference_value):
    average_distance = unrolled_distances[unrolled_distances['id_start'] == reference_value]['distance'].mean()

    threshold_lower = average_distance - (average_distance * 0.1)
    threshold_upper = average_distance + (average_distance * 0.1)

    filtered_ids = sorted(unrolled_distances[(unrolled_distances['distance'] >= threshold_lower) & (unrolled_distances['distance'] <= threshold_upper)]['id_start'].unique())

    return filtered_ids

# Example Usage:
reference_value = 1001400
result_question_3 = find_ids_within_ten_percentage_threshold(unrolled_distances, reference_value)
print(result_question_3)


[]


[]


# Question 4: Calculate Toll Rate
Create a function calculate_toll_rate that takes the DataFrame created in Question 2 as input and calculates toll rates based on vehicle types.

The resulting DataFrame should add 5 columns to the input DataFrame: moto, car, rv, bus, and truck with their respective rate coefficients. The toll rates should be calculated by multiplying the distance with the given rate coefficients for each vehicle type:

In [46]:
import pandas as pd

def calculate_toll_rate(unrolled_distances):
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

    for vehicle_type in rate_coefficients.keys():
        unrolled_distances[vehicle_type] = unrolled_distances['distance'] * rate_coefficients[vehicle_type]

    return unrolled_distances

toll_rates = calculate_toll_rate(unrolled_distances)
print(toll_rates)


      id_start   id_end  distance   moto    car    rv    bus  truck
0      1001400  1001402      19.4  15.52  23.28  29.1  42.68  69.84
1      1001400  1001404       0.0   0.00   0.00   0.0   0.00   0.00
2      1001400  1001406       0.0   0.00   0.00   0.0   0.00   0.00
3      1001400  1001408       0.0   0.00   0.00   0.0   0.00   0.00
4      1001400  1001410       0.0   0.00   0.00   0.0   0.00   0.00
...        ...      ...       ...    ...    ...   ...    ...    ...
1801   1004356  1001470       0.0   0.00   0.00   0.0   0.00   0.00
1802   1004356  1001472       0.0   0.00   0.00   0.0   0.00   0.00
1803   1004356  1001488       8.0   6.40   9.60  12.0  17.60  28.80
1804   1004356  1004354       4.0   3.20   4.80   6.0   8.80  14.40
1805   1004356  1004355       0.0   0.00   0.00   0.0   0.00   0.00

[1806 rows x 8 columns]


# Question 5: Calculate Time-Based Toll Rates

In [48]:
import pandas as pd

def calculate_time_based_toll_rates(toll_rates):
    toll_rates['start_day'] = toll_rates['end_day'] = toll_rates['start_time'] = toll_rates['end_time'] = ""

    time_ranges = {
        '00:00:00-10:00:00': 0.8,
        '10:00:00-18:00:00': 1.2,
        '18:00:00-23:59:59': 0.8,
        'weekend': 0.7
    }

    for time_range, discount_factor in time_ranges.items():
        if time_range != 'weekend':
            start_time, end_time = time_range.split('-')
            mask = toll_rates['start_time'].between(start_time, end_time, inclusive='both')
        else:
            mask = toll_rates['start_time'] != ""

        toll_rates.loc[mask, ['start_day', 'end_day', 'start_time', 'end_time']] = ['', '', '', '']
        toll_rates.loc[mask, ['moto', 'car', 'rv', 'bus', 'truck']] *= discount_factor

    return toll_rates

time_based_toll_rates = calculate_time_based_toll_rates(toll_rates)
print(time_based_toll_rates)


      id_start   id_end  distance   moto    car    rv    bus  truck start_day  \
0      1001400  1001402      19.4  15.52  23.28  29.1  42.68  69.84             
1      1001400  1001404       0.0   0.00   0.00   0.0   0.00   0.00             
2      1001400  1001406       0.0   0.00   0.00   0.0   0.00   0.00             
3      1001400  1001408       0.0   0.00   0.00   0.0   0.00   0.00             
4      1001400  1001410       0.0   0.00   0.00   0.0   0.00   0.00             
...        ...      ...       ...    ...    ...   ...    ...    ...       ...   
1801   1004356  1001470       0.0   0.00   0.00   0.0   0.00   0.00             
1802   1004356  1001472       0.0   0.00   0.00   0.0   0.00   0.00             
1803   1004356  1001488       8.0   6.40   9.60  12.0  17.60  28.80             
1804   1004356  1004354       4.0   3.20   4.80   6.0   8.80  14.40             
1805   1004356  1004355       0.0   0.00   0.00   0.0   0.00   0.00             

     end_day start_time end